In [4]:
# chatbot
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# llm
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

import os
import gradio as gr
import os
from playsound import playsound
import openai
from openai import OpenAI
import playsound
import speech_recognition as sr
import pandas as pd

api_key = "내 키"

openai.api_key = api_key
client = OpenAI


보안유지 ( api_key는 유출상, github 올리지 말기)

In [5]:
data = pd.read_csv("./data.csv")
data
data.loc[3] = ["김태현", 1,'가정형편, 주거관련 위기']
data

,이름,주민등록상,문제
0,김태경,혼자,건강보험료 연체
1,주예서,혼자,건강 문제
2,강민지,2,공과급 체납
3,김태현,1,"가정형편, 주거관련 위기"


In [6]:
a=2

# 음성인식 모듈
def recognize_speech(timeout=60):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source) 
        print("음성 인식 대기중")
        audio = recognizer.listen(source,timeout=timeout)

    try:
        text = recognizer.recognize_google(audio, language = "ko")
        print(text)
        return text 
    except sr.UnknownValueError:
        print("인식할 수 없습니다.")
    except sr.RequestError as e:
        print("인식에 문제가 있습니다.", e)


# tts 모듈
def tts_module(texts):
    voices = ['nova']
    for i, text in enumerate(texts):
        speech_file_path = f'text_{i}.mp3'
        response = openai.audio.speech.create(
            model="tts-1",
            voice=voices[i],
            input=" "+text
        )
        response.stream_to_file(speech_file_path)
        playsound.playsound(f"./text_{i}.mp3", block=True)
        os.remove(f'./text_{i}.mp3')


# 텍스트 생성 모듈
chat_ai = ChatOpenAI(openai_api_key=api_key, temperature=0.5, model="gpt-4-1106-preview")

chat_template = f"""필수 : 나는 복지상담을 진행하는 음성봇 이다.
한번에 한가지 질문만 한다. 
공손하고 예의바르게 말을 해야한다. 
전화통화를 하는 것처럼 대화를 해야한다. 
답변은 2줄이내로 진행한다. 
해당 전화는 본 챗봇이 먼저 전화를 건 상황이다.
총 6번이상의 대화가 오고가도록 한다.
음성 봇이라고 자기소개는 하지 않는다


0. 상대방의 이름은 {data["이름"][a]}이다. 해당 이름으로 대화를 한다.

1. 첫 대화에 있어서 인사는 하지 않는다

1. 상대방의 문제 상황은 "{data["문제"][a]}" 이다. 보건복지부 자료에 따른 해당 문제 상황에 있는게 맞는지 친절하게 말하며 사실인지 확인한다.

2. 사실이 아니라는 의미의 대답을 하면 상담이 필요 없는지 묻는다.

3. 상담이 필요 없다고 대답하면 친절한 어투로 대화를 종료한다.

4. 사실이 맞다면 {data["문제"][a]}에 대해 구체적으로 질문한다.

5. {data["문제"][a]}에 대해 구체적인 상황은 어떠한지 질문을 한다.

6. 상대방과 대화도중 새로운 문제나 상황이 발생했으면 그 문제,상황에 대해서 다시 구체적으로 질문한다.

7. 상대방의 문제에 대해 어느정도 질문을 했으면 주변에 도움을 요청할만한 보호자가 없는지 물어본다.

8. 질문이 최소 5번 이상 오가고 난 이후에, 상대방의 어려움에 관련된 복지 서비스를 추천해준다.

9. 서비스를 모두 추천해준 후 추가 상담을 원한다면 대면 상담을 진행할지 비대면 상담을 진행할지 묻고, 추가 상담을 원하지 않는다면 친절한 어투로 대화를 종료한다.

"""

history = []  # 대화 기록 초기화
first_ment = f'네 안녕하세요. 맞춤형 복지 음성 서비스 아이 입니다. 통화에 앞서 본 통화는 녹취된다는 사실 말씀드립니다'
# tts_module([first_ment])
print(first_ment)


while True:
    # 사용자 입력 받기
    # message = recognize_speech()
    message = input()
    print("User: ",message)
    

    # 종료하겠습니다를 입력하면 최종 요약 출력
    if message == "종료하겠습니다":
        print(history)
        # tts_module(["네 지금까지 복지 음성 상담서비스 아이였습니다 감사합니다"])
        break

    history_langchain_format = []
    # 프롬프트 추가
    history_langchain_format.append(SystemMessage(content=chat_template))
    # 이전 대화 기억
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = chat_ai(history_langchain_format)
    texts = [gpt_response.content]
    # tts_module(texts)
    
    print("AI:", gpt_response.content)
    history.append((message, gpt_response.content))



네 안녕하세요. 맞춤형 복지 음성 서비스 아이 입니다. 통화에 앞서 본 통화는 녹취된다는 사실 말씀드립니다
User:  안녕하세요
AI: 안녕하세요, 강민지 님. 보건복지부 자료에 따르면 공과급 체납이 있는 것으로 확인되었는데, 이 부분에 대해 맞는 정보인지 확인해주실 수 있으신가요?
User:  종료하겠습니다
[('안녕하세요', '안녕하세요, 강민지 님. 보건복지부 자료에 따르면 공과급 체납이 있는 것으로 확인되었는데, 이 부분에 대해 맞는 정보인지 확인해주실 수 있으신가요?')]


In [51]:
a=2

# 음성인식 모듈
def recognize_speech(timeout=60):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source) 
        print("음성 인식 대기중")
        audio = recognizer.listen(source,timeout=timeout)

    try:
        text = recognizer.recognize_google(audio, language = "ko")
        print(text)
        return text 
    except sr.UnknownValueError:
        print("인식할 수 없습니다.")
    except sr.RequestError as e:
        print("인식에 문제가 있습니다.", e)


# tts 모듈
def tts_module(texts):
    voices = ['nova']
    for i, text in enumerate(texts):
        speech_file_path = f'text_{i}.mp3'
        response = openai.audio.speech.create(
            model="tts-1",
            voice=voices[i],
            input=" "+text
        )
        response.stream_to_file(speech_file_path)
        playsound.playsound(f"./text_{i}.mp3", block=True)
        os.remove(f'./text_{i}.mp3')


# 텍스트 생성 모듈
chat_ai = ChatOpenAI(openai_api_key=api_key, temperature=0.5, model="gpt-4-1106-preview")

chat_template = f"""필수 : 나는 복지상담을 진행하는 음성봇 이다.
한번에 한가지 질문만 한다. 
공손하고 예의바르게 말을 해야한다. 
전화통화를 하는 것처럼 대화를 해야한다. 
답변은 2줄이내로 진행한다. 
해당 전화는 본 챗봇이 먼저 전화를 건 상황이다.
총 6번이상의 대화가 오고가도록 한다.
음성 봇이라고 자기소개는 하지 않는다

 """

history = []  # 대화 기록 초기화
first_ment = f'네 안녕하세요. 맞춤형 복지 음성 서비스 아이 입니다. 통화에 앞서 본 통화는 녹취된다는 사실 말씀드립니다'
# tts_module([first_ment])
print(first_ment)


while True:
    # 사용자 입력 받기
    # message = recognize_speech()
    message = input()
    print("User: ",message)
    

    # 종료하겠습니다를 입력하면 최종 요약 출력
    if message == "종료하겠습니다":
        print(history)
        # tts_module(["네 지금까지 복지 음성 상담서비스 아이였습니다 감사합니다"])
        break

    history_langchain_format = []
    # 프롬프트 추가
    history_langchain_format.append(SystemMessage(content=chat_template))
    # 이전 대화 기억
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = chat_ai(history_langchain_format)
    texts = [gpt_response.content]
    # tts_module(texts)
    
    print("AI:", gpt_response.content)
    history.append((message, gpt_response.content))



네 안녕하세요. 맞춤형 복지 음성 서비스 아이 입니다. 통화에 앞서 본 통화는 녹취된다는 사실 말씀드립니다
User:  안녕하세요
AI: 안녕하세요, 좋은 하루 보내고 계신가요? 오늘 도와드릴 일이 있으신가요?
User:  안녕하세요
AI: 네, 안녕하세요. 어떤 도움을 드릴까요?


In [10]:
# print(history)

full_dialogue = '\n'.join([f'{user_msg}\n{ai_response}' for user_msg, ai_response in history])

# 결과 출력
print(full_dialogue)

네 안녕하세요
안녕하세요 강민지 님, 보건복지부 자료에 따르면 공과급 체납에 대한 상담이 필요하신 것으로 나타나는데, 맞으신가요?
네 맞아요
공과급 체납에 대해 어떤 부분에서 도움이 필요하신지 구체적으로 말씀해 주실 수 있으신가요?


In [7]:
# 단순 카운트 함수

def simple_count(tokens, n): # 토큰화 된 candidate 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, n)) #문장에서 n-gram을 카운

# count_clip 이름을 가진 함수 구현

def count_clip(candidate, reference_list, n):
    cnt_ca = simple_count(candidate, n)
    # Ca 문장에서 n-gram 카운트
    temp = dict()

    for ref in reference_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트

        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]

    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
     }

# 보정된 정밀도를 연산하는 함수를 modified_precision란 이름의 함수로 구현

def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(candidate, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

def closest_ref_length(candidate, reference_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

# brevity_penalty를 구현

def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

In [8]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

- candidate1 = 기존 chat gpt 
- candidate2 = 우리가 prompting 한 gpt
- references = 실제 시나리오

In [9]:
from collections import Counter
import numpy as np
from nltk import ngrams
import nltk.translate.bleu_score as bleu

In [10]:
can1=[]
can2=[]

In [15]:
candidate1 = "네, 안녕하세요. 어떤 도움을 드릴까요?"
candidate2 = "안녕하세요,  복지 상담센터 '아이'입니다. 건강 문제로 어려움을 겪고 계신다고 들었습니다. 혹시 제가 들은 정보가 맞는지 확인해 드리기 위해 전화드렸어요. 맞으신가요?"
references = [
'네 안녕하세요. 곡성읍사무소 맞춤형 복지팀 주무관 OOO입니다. 제가 전화드린 이유는 정기적으로 가정 형편이나 주거 관련해서 위기 사항이 있으신 분에 대해서 복지부에서 정보가 내려와서 정기적으로 전화를 드리고 있습니다. '
,'그래서 저희가 복지부에서 주기적으로 가정 형편이 어려운 가정에 대해서 전화를 드리고 있는데 혹시 안 좋은 일이나 문제가 있으시간 모르겠네요.' 
,'안녕하세요. 읍사무소 맞춤형 복지팀 주무관 OOO라고 합니다. 혹시 지금 위기 상황으로 확인이 되셔서 저희가 연락을 드렸는데요. 어떤 어려운 점 있으신가요?'
,'안녕하세요. 저는 곡성사무소 맞춤형 복지팀 주무관 OOO입니다. 할아버지가 몸이 안 좋으신 것 같아서 전화를 드렸어요. 어디 안 좋은 데 있으세요?'
,'안녕하세요. 여기는 양평읍사무소 찾아오는 복지팀에 OOO인데요. 혹시 OOO 님 맞으시죠? 다름이 아니고 저희가 이번에 복지 사각지대 발굴 대상자로 통신비 체납으로 명단을 받았어요.' 
,'네 안녕하세요. 선생님 여기 교문1동 행정복지센터 복지팀 주무관 OOO입니다. 혹시 실례지만 OOO님 맞으실까요?'
,'선생님 지금 저희가 확인을 해 보니까 지금 건강보험료가 조금 많이 체납이 되어 있는 걸로 조회가 되시는데 혹시 알고 계세요?'
,'선생님 지금 공과금이 체납된 걸로 나오시는데 혹시 체납되시기까지의 그런 과정에 있어서 예를 들어서 실직을 하셨다든지 고용에 불안정하시다든지 그런 어려움이 있으실까요?'
]

can1_score = bleu_score(candidate1.split(),list(map(lambda ref: ref.split(), references)))
# can1.append(can1_score)
print("chat gpt score : " ,can1_score)

can2_score = bleu_score(candidate2.split(),list(map(lambda ref: ref.split(), references)))
# can2.append(can2_score)
print("our gpt score : " ,can2_score)

chat gpt score :  0.16056239100113953
our gpt score :  0.668740304976422


In [25]:
candidate1 = "안녕! [친구/가족 이름]! 어떻게 지내?"
candidate2 = "안녕하세요, 복지 상담센터입니다. 가정형편과 주거관련 위기 상황으로 어려움을 겪고 계신다고 들어서 확인차 연락드렸습니다. 혹시 맞으신가요?"
references = [
'네 안녕하세요. 곡성읍사무소 맞춤형 복지팀 주무관 OOO입니다. 제가 전화드린 이유는 정기적으로 가정 형편이나 주거 관련해서 위기 사항이 있으신 분에 대해서 복지부에서 정보가 내려와서 정기적으로 전화를 드리고 있습니다. '
,'그래서 저희가 복지부에서 주기적으로 가정 형편이 어려운 가정에 대해서 전화를 드리고 있는데 혹시 안 좋은 일이나 문제가 있으시간 모르겠네요.' 
,'안녕하세요. 읍사무소 맞춤형 복지팀 주무관 OOO라고 합니다. 혹시 지금 위기 상황으로 확인이 되셔서 저희가 연락을 드렸는데요. 어떤 어려운 점 있으신가요?'
,'안녕하세요. 저는 곡성사무소 맞춤형 복지팀 주무관 OOO입니다. 할아버지가 몸이 안 좋으신 것 같아서 전화를 드렸어요. 어디 안 좋은 데 있으세요?'
,'안녕하세요. 여기는 양평읍사무소 찾아오는 복지팀에 OOO인데요. 혹시 OOO 님 맞으시죠? 다름이 아니고 저희가 이번에 복지 사각지대 발굴 대상자로 통신비 체납으로 명단을 받았어요.' 
,'네 안녕하세요. 선생님 여기 교문1동 행정복지센터 복지팀 주무관 OOO입니다. 혹시 실례지만 OOO님 맞으실까요?'
,'선생님 지금 저희가 확인을 해 보니까 지금 건강보험료가 조금 많이 체납이 되어 있는 걸로 조회가 되시는데 혹시 알고 계세요?'
,'선생님 지금 공과금이 체납된 걸로 나오시는데 혹시 체납되시기까지의 그런 과정에 있어서 예를 들어서 실직을 하셨다든지 고용에 불안정하시다든지 그런 어려움이 있으실까요?'
]
can1_score = bleu_score(candidate1.split(),list(map(lambda ref: ref.split(), references)))
can1.append(can1_score)
print("chat gpt score : " ,can1_score)

can2_score = bleu_score(candidate2.split(),list(map(lambda ref: ref.split(), references)))
can2.append(can2_score)
print("our gpt score : " ,can2_score)

chat gpt score :  0.20189651799465538
our gpt score :  0.392814650900513


In [26]:
print(can1)
print(can2)

[0.36720562698935927, 0.16056239100113953, 0.16056239100113953, 0.4402274324685413, 0.4402274324685413, 0.4165926999433137, 0.4165926999433137, 0.13501633901742352, 0.19896894256029013, 0.20189651799465538]
[0.36821398145189993, 0.7476743906106103, 0.668740304976422, 0.6494854111739939, 0.7146704964214272, 0.392814650900513, 0.668740304976422, 0.6044142795544439, 0.668740304976422, 0.392814650900513]
